> **Credits:** Based on [this](https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7963) amazing notebook.

OK. Maybe not **all** you need.
<br>
But they improve `LightGBM`!
_____

In [10]:


from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from typing_extensions import Self
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
from cycler import cycler
from IPython.display import display
import datetime
import scipy.stats
import warnings
from colorama import Fore, Back, Style
import gc
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibrationDisplay
from lightgbm import LGBMClassifier, log_evaluation

plt.rcParams['axes.facecolor'] = '#0057b8'  # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
plt.rcParams['text.color'] = 'w'

# Add
print(os.getcwd())

C:\Users\jackm


In [5]:
##################
# RUN Parameters
###

INFERENCE = True  # True for LOCAL RUN   + needed for DART processing

ONLY_FIRST_FOLD = False  # False for LOCAL RUN

USE_ALL_FEATURES = False  # True for LOCAL RUN

USE_LGB_WITH_DART = True  # DART will make training run VERY SLOWLY

# - - - - - - - - - - - - - - - - - - - - - - - - -

# Select one of: 'last_set' or 'top_impt_set' or 'minimal_set'
IF_NOT_USE_ALL_FEATURES_THEN_USE = 'last_set'

# The data contains 1-13 monthly data profiles for each customer. We will use the first, last and nth1 and nth2 for modelling.
# Valid values for the next two variables are 0-12 where 0 is the first monthly data profile and 12 is the last (the 13th).
# IMPT: If the requested nth data profile does not exist for a customer then the "last" data profile will be used instead.
# IMPT: Use negative number references to have FEWER missed values for customers with less than 13 monthly data profiles.
# The selected monthly profile to be used for nth1 (was 4, -3, -9=4)
VALUE_FOR_MONTH_NTH1 = -9
# The selected monthly profile to be used for nth2 (was 8, -2, -5=8)
VALUE_FOR_MONTH_NTH2 = -5
# If True then include "divide by last" (first/last, nth1/last, nth2/last) features
PCT_CALCULATION_DIVIDE_BY_LAST = True
# If True then include "divide by next" (first/nth1, nth1/nth2, nth2/last) features
PCT_CALCULATION_DIVIDE_BY_NEXT = False

DROP_SOME_TRAIN_DATA = False  # Causes errors - not yet working

# FILL NAN VALUE
NAN_VALUE = 0  # WAS -127


# Preprocessing

In [6]:
import pyarrow as pa
from pyarrow.parquet import ParquetFile
import thop
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision

import optuna
import numpy as np
import os
import pandas as pd
# max_depth = range(3,12)


# Num_leaves should be maximum of 2^(max_depth)

# min)data_in_leaf-> number of observations that fir the decision
# criteria

# n_estimators -> number of decision trees
# learning_rate -> step size param of gradient descent


DIR = ".."
BATCHSIZE = 128
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


# Added


pf = ParquetFile(
    f'../input/amex-data-integer-dtypes-parquet-format/train.parquet')
first_ten_rows = next(pf.iter_batches(batch_size=10))
df = pa.Table.from_batches([first_ten_rows]).to_pandas()

# These are not used in original code below
df.drop(columns=['customer_ID', 'S_2'], inplace=True)
all_features = df.columns
print(str(len(all_features)))
print(str(all_features))

cat_cols = ['Balance 30', 'Balance 38', 'Delinquency 63', 'Delinquency 64', 'Delinquency 66', 'Delinquency 68',
            'Delinquency 114', 'Delinquency 116', 'Delinquency 117', 'Delinquency 120', 'Delinquency 126', 'Target']
cols = [col for col in df.columns if (
    col.startswith(('B'))) & (col not in cat_cols[:-1])]
# DROP R_ cols TO SAVE RAM for this test
df.drop(columns=cols, inplace=True)

all_features = df.columns
print(str(len(all_features)))
print(str(all_features))

del pf, first_ten_rows, df
gc.collect()


FileNotFoundError: [WinError 3] Failed to open local file '../input/amex-data-integer-dtypes-parquet-format/train.parquet'. Detail: [Windows error 3] The system cannot find the path specified.


In [8]:
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

def lgb_amex_metric(y_true, y_pred):
    """The competition metric with lightgbm's calling convention"""
    return ('amex',
            amex_metric(y_true, y_pred),
            True)

In [9]:
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]

target = pd.read_csv(
    '../input/amex-default-prediction/train_labels.csv').target.values

print(f"shape of target: {target.shape}")

gc.collect()

train = pd.read_parquet(
    f'../input/amex-data-integer-dtypes-parquet-format/train.parquet')

print(f"shape of training data: {train.shape}")


FileNotFoundError: [Errno 2] No such file or directory: '../input/amex-default-prediction/train_labels.csv'

In [ ]:
# ORIG
print('Original feature sets...')
features_avg = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18',
                'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40',
                'B_41', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_50', 'D_51', 'D_53', 'D_54', 'D_55',
                'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_66', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77',
                'D_78', 'D_80', 'D_82', 'D_84', 'D_86', 'D_91', 'D_92', 'D_94', 'D_96', 'D_103', 'D_104', 'D_108', 'D_112', 'D_113', 'D_114',
                'D_115', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_129', 'D_131',
                'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2',
                'R_3', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11', 'R_14', 'R_15', 'R_16', 'R_17', 'R_20', 'R_21', 'R_22', 'R_24', 'R_26', 'R_27',
                'S_3', 'S_5', 'S_6', 'S_7', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_18', 'S_22', 'S_23', 'S_25', 'S_26']

features_min = ['B_2', 'B_4', 'B_5', 'B_9', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_19', 'B_20', 'B_28', 'B_29', 'B_33', 'B_36', 'B_42', 'D_39',
                'D_41', 'D_42', 'D_45', 'D_46', 'D_48', 'D_50', 'D_51', 'D_53', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_62', 'D_70', 'D_71',
                'D_74', 'D_75', 'D_78', 'D_83', 'D_102', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_128', 'D_132', 'D_140',
                'D_141', 'D_144', 'D_145', 'P_2', 'P_3', 'R_1', 'R_27', 'S_3', 'S_5', 'S_7', 'S_9', 'S_11', 'S_12', 'S_23', 'S_25']

features_max = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18',
                'B_19', 'B_21', 'B_23', 'B_24', 'B_25', 'B_29', 'B_30', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_42', 'D_39', 'D_41', 'D_42',
                'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_52', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_63',
                'D_64', 'D_65', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_91', 'D_102', 'D_105',
                'D_107', 'D_110', 'D_111', 'D_112', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125',
                'D_126', 'D_128', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_138', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145',
                'P_2', 'P_3', 'P_4',
                'R_1', 'R_3', 'R_5', 'R_6', 'R_7', 'R_8', 'R_10', 'R_11', 'R_14', 'R_17', 'R_20', 'R_26', 'R_27',
                'S_3', 'S_5', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27'
                ]

features_last_PATRIAL = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8',  # 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18',
                         # 'B_26', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38',
                         'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25',
                         # 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51',
                         'B_39', 'B_40', 'B_41', 'B_42', 'D_39', 'D_41', 'D_42',
                         # 'D_60', 'D_61', 'D_62', 'D_63', 'D_64', 'D_65', 'D_69', 'D_70', 'D_71',
                         'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59',
                         # 'D_80', 'D_81', 'D_82', 'D_83', 'D_86', 'D_91', 'D_96', 'D_105', 'D_106',
                         'D_72', 'D_73', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79',
                         # 'D_124', 'D_125', 'D_126', 'D_127', 'D_130', 'D_131', 'D_132', 'D_133',
                         'D_112', 'D_114', 'D_119', 'D_120', 'D_121', 'D_122',
                         # 'P_3', 'P_4', 'R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7',
                         'D_134', 'D_138', 'D_140', 'D_141', 'D_142', 'D_145', 'P_2',
                         # 'R_15', 'R_19', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7',
                         'R_8', 'R_9', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14',
                         # 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27'
                         'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_16', 'S_19',
                         ]

features_last = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18',
                 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38',
                 'B_39', 'B_40', 'B_41', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51',
                 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63', 'D_64', 'D_65', 'D_69', 'D_70', 'D_71',
                 'D_72', 'D_73', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_86', 'D_91', 'D_96', 'D_105', 'D_106',
                 'D_112', 'D_114', 'D_119', 'D_120', 'D_121', 'D_122', 'D_124', 'D_125', 'D_126', 'D_127', 'D_130', 'D_131', 'D_132', 'D_133',
                 'D_134', 'D_138', 'D_140', 'D_141', 'D_142', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7',
                 'R_8', 'R_9', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_19', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7',
                 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_16', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27'
                 ]

# BASED ON "gml AMEX Default Prediction EDA & LGBM Baseline" RUNS USING A SINGLE TYPES OF FEATURES (v6,v14,v13,v15,v12)
# If kept to 30 then will allow inference to be run within Kaggle RAM limits
features_top_impt = ['D_43', 'D_60', 'D_46', 'D_45', 'D_39', 'D_48', 'D_47', 'D_53', 'D_59', 'D_44',  # 'D_71', 'D_61', 'D_75', 'D_121', 'D_62',
                     # 'B_19', 'B_15', 'B_10', 'B_6', 'B_13',
                     'B_9', 'B_5', 'B_3', 'B_26', 'B_40', 'B_4', 'B_8',
                     'P_2', 'P_3', 'P_4',
                     'R_3', 'R_27', 'R_1', 'R_2', 'R_6',  # 'R_10', 'R_5', 'R_21', 'R_20', 'R_7',
                     'S_22', 'S_24', 'S_27', 'S_9', 'S_5',  # 'S_3', 'S_7', 'S_13', 'S_23', 'S_12',
                     #'_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_',
                     ]

features_minimal = ['D_43', 'D_60', 'D_46',  # 'D_45', 'D_39', 'D_48', 'D_47', 'D_53', 'D_59', 'D_44', #'D_71', 'D_61', 'D_75', 'D_121', 'D_62',
                    # 'B_26', 'B_40', 'B_4', 'B_8', #'B_19', 'B_15', 'B_10', 'B_6', 'B_13',
                    'B_9', 'B_5', 'B_3',
                    'P_2', 'P_3', 'P_4',
                    'R_3', 'R_27', 'R_1',  # 'R_2', 'R_6', #'R_10', 'R_5', 'R_21', 'R_20', 'R_7',
                    'S_22', 'S_24', 'S_27',  # 'S_9', 'S_5',# 'S_3', 'S_7', 'S_13', 'S_23', 'S_12',
                    #'_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_',
                    ]

features_tiny = ['D_43',  # 'D_60', 'D_46', #'D_45', 'D_39', 'D_48', 'D_47', 'D_53', 'D_59', 'D_44', #'D_71', 'D_61', 'D_75', 'D_121', 'D_62',
                 'B_9',  # 'B_5', 'B_3', #'B_26', 'B_40', 'B_4', 'B_8', #'B_19', 'B_15', 'B_10', 'B_6', 'B_13',
                 'P_2',  # 'P_3', 'P_4',
                 'R_3',  # 'R_27', 'R_1', #'R_2', 'R_6', #'R_10', 'R_5', 'R_21', 'R_20', 'R_7',
                 'S_22',  # 'S_24', 'S_27', #'S_9', 'S_5',# 'S_3', 'S_7', 'S_13', 'S_23', 'S_12',
                 #'_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_',
                 ]

print('features_avg: ' + str(len(features_avg)))
print('features_min: ' + str(len(features_min)))
print('features_max: ' + str(len(features_max)))
print('features_last: ' + str(len(features_last)))
print('features_last_PATRIAL: ' + str(len(features_last_PATRIAL)))
print('features_top_impt: ' + str(len(features_top_impt)))
print('features_minimal: ' + str(len(features_minimal)))
print('features_tiny: ' + str(len(features_tiny)))
print()

# REVISED set of features to be used:
# features_to_use = features_last_PATRIAL
# features_to_use = features_last
# features_to_use = all_features
#features_to_use = features_top_impt
# features_to_use = features_max
if IF_NOT_USE_ALL_FEATURES_THEN_USE == 'last_set':
    features_to_use = features_last
    print('IF_NOT_USE_ALL_FEATURES_THEN_USE: last_set')
elif IF_NOT_USE_ALL_FEATURES_THEN_USE == 'minimal_set':
    features_to_use = features_minimal
    print('IF_NOT_USE_ALL_FEATURES_THEN_USE: minimal_set')
elif IF_NOT_USE_ALL_FEATURES_THEN_USE == 'tiny_set':
    features_to_use = features_tiny
    print('IF_NOT_USE_ALL_FEATURES_THEN_USE: tiny_set')
else:  # Use the smaller 'top_impt_set' by defalult
    features_to_use = features_top_impt
    print('IF_NOT_USE_ALL_FEATURES_THEN_USE: top_impt_set')

print(str(len(features_to_use)))
print(str(features_to_use))
print()

print('Revised features sets...')
features_avg = features_to_use
features_min = features_to_use
features_max = features_to_use
features_last = features_to_use
print(str(len(features_avg)))
print(str(len(features_min)))
print(str(len(features_max)))
print(str(len(features_last)))


Original feature sets...
features_avg: 140
features_min: 63
features_max: 124
features_last: 142
features_last_PATRIAL: 63
features_top_impt: 30
features_minimal: 15
features_tiny: 5

IF_NOT_USE_ALL_FEATURES_THEN_USE: last_set
142
['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63', 'D_64', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_86', 'D_91', 'D_96', 'D_105', 'D_106', 'D_112', 'D_114', 'D_119', 'D_120', 'D_121', 'D_122', 'D_124', 'D_125', 'D_126', 'D_127', 'D_130', 'D_131', 'D_132', 'D

In [ ]:
select_cols = [col for col in train.columns if (col.startswith(('P_2')))]
features = [f for f in train.columns if f != 'customer_ID' and f != 'target']
print(str(len(select_cols)))
train[select_cols]
print(len(train[features]))


1
5531451


In [ ]:
# Added

from pyarrow.parquet import ParquetFile
import pyarrow as pa

pf = ParquetFile(
    f'../input/amex-data-integer-dtypes-parquet-format/train.parquet')
first_ten_rows = next(pf.iter_batches(batch_size=10))
df = pa.Table.from_batches([first_ten_rows]).to_pandas()

# These are not used in original code below
df.drop(columns=['customer_ID', 'S_2'], inplace=True)
all_features = df.columns
print(str(len(all_features)))
print(str(all_features))

cat_cols = ['Balance 30', 'Balance 38', 'Delinquency 63', 'Delinquency 64', 'Delinquency 66', 'Delinquency 68',
            'Delinquency 114', 'Delinquency 116', 'Delinquency 117', 'Delinquency 120', 'Delinquency 126', 'Target']
cols = [col for col in df.columns if (
    col.startswith(('B'))) & (col not in cat_cols[:-1])]
# DROP R_ cols TO SAVE RAM for this test
df.drop(columns=cols, inplace=True)

all_features = df.columns
print(str(len(all_features)))
print(str(all_features))

del pf, first_ten_rows, df
gc.collect()


188
Index(['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42',
       'D_43',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143',
       'D_144', 'D_145'],
      dtype='object', length=188)
148
Index(['P_2', 'D_39', 'R_1', 'S_3', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45',
       'R_2',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143',
       'D_144', 'D_145'],
      dtype='object', length=148)


0

# Process and feature engineering 

---

In [47]:
# ADAPTED FROM ---> https://www.kaggle.com/code/glaskosk/xgboost-starter-0-793?scriptVersionId=101352220

from hashlib import new


def process_and_feature_engineer(df):
    # FEATURE ENGINEERING FROM
    # https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID', 'S_2']]
    cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117",
                    "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
    num_features = [col for col in all_cols if col not in cat_features]
    print(str(len(num_features)))

    # DEFAULT values to use if some below are NaN << [0]=FIRST and [-1]=LAST >>
    NO_NAN_df_nth = df.groupby("customer_ID")[num_features].nth([-1])
    # REDUNDANT DATA--> NO_NAN_df_nth = df.groupby("customer_ID")[num_features].agg(['last']) ### MAY be BETTER to use last instead of first for missing instances
    # REDUNDANT DATA--> NO_NAN_df_nth.columns = ['_'.join(x) for x in NO_NAN_df_nth.columns]  ### INCLUDE this line ONLY IF .agg(['last'])
    gc.collect()

    print('VALUE_FOR_MONTH_NTH1: ' + str(VALUE_FOR_MONTH_NTH1))
    # 0 to N;   #.dropna()  Keep NaNs  e.g. if nth = 15 then all are NaNs
    test_df_nth = df.groupby("customer_ID")[
        num_features].nth([VALUE_FOR_MONTH_NTH1])
    test_df_nth = test_df_nth.combine_first(NO_NAN_df_nth)

    test_df_nth_col_names = test_df_nth.columns
    test_df_nth_col_names = test_df_nth_col_names + '_nth1'
    test_df_nth.columns = test_df_nth_col_names
    gc.collect()
    print('1) Calculated _nth1')

    print('VALUE_FOR_MONTH_NTH2: ' + str(VALUE_FOR_MONTH_NTH2))
    # 0 to N;   #.dropna()  Keep NaNs  e.g. if nth = 15 then all are NaNs
    test_df_nth2 = df.groupby("customer_ID")[
        num_features].nth([VALUE_FOR_MONTH_NTH2])
    test_df_nth2 = test_df_nth2.combine_first(NO_NAN_df_nth)
    del NO_NAN_df_nth
    gc.collect()

    test_df_nth2_col_names = test_df_nth2.columns
    test_df_nth2_col_names = test_df_nth2_col_names + '_nth2'
    test_df_nth2.columns = test_df_nth2_col_names
    print('2) Calculated _nth2')

#     test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last']) ### ORIG
    test_num_agg = df.groupby("customer_ID")[num_features].agg(
        ['first', 'last', 'min', 'max', 'mean', 'median', 'std'])  # , 'median', 'std'
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    print('3) Calculated test_num_agg (e.g. first,last,min,max,mean,median,std)')

    test_num_agg = pd.concat([test_df_nth, test_df_nth2, test_num_agg], axis=1)
    del test_df_nth, test_df_nth2
    gc.collect()

    # Lag Features
    for col in test_num_agg:
        new_cols = {}
        # "divide by last" (first/last, nth1/last, nth2/last)
        if PCT_CALCULATION_DIVIDE_BY_LAST == True:
            # if 'last' in col and col.replace('last', 'first') in test_num_agg:
            #test_num_agg[col + '_lag_sub'] = test_num_agg[col] - test_num_agg[col.replace('last', 'first')]
            if 'first' in col and col.replace('first', 'last') in test_num_agg:
                new_cols[col + 'div_last'] = (test_num_agg[col.replace(
                    'first', 'last')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
                #test_num_agg[col + '_div_last'] = (test_num_agg[col.replace('first', 'last')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
            if 'nth1' in col and col.replace('nth1', 'last') in test_num_agg:
                new_cols[col + '_div_last'] = (test_num_agg[col.replace(
                    'nth1', 'last')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
                #test_num_agg[col + '_div_last'] = (test_num_agg[col.replace('nth1', 'last')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
            if 'nth2' in col and col.replace('nth2', 'last') in test_num_agg:
                new_cols[col + '_div_last'] = (test_num_agg[col.replace(
                    'nth2', 'last')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
                #test_num_agg[col + '_div_last'] = (test_num_agg[col.replace('nth2', 'last')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)

        # "divide by next" (first/nth1, nth1/nth2, nth2/last)
        if PCT_CALCULATION_DIVIDE_BY_NEXT == True:
            if 'first' in col and col.replace('first', 'nth1') in test_num_agg:
                new_cols[col + '_div_nth1'] = (test_num_agg[col.replace(
                    'first', 'nth1')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
                #test_num_agg[col + '_div_nth1'] = (test_num_agg[col.replace('first', 'nth1')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
            if 'nth1' in col and col.replace('nth1', 'nth2') in test_num_agg:
                new_cols[col + '_div_nth2'] = (test_num_agg[col.replace(
                    'nth1', 'nth2')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
                #test_num_agg[col + '_div_nth2'] = (test_num_agg[col.replace('nth1', 'nth2')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
            if 'nth2' in col and col.replace('nth2', 'last') in test_num_agg:
                new_cols[col + '_div_last'] = (test_num_agg[col.replace(
                    'nth2', 'last')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
                #test_num_agg[col + '_div_last'] = (test_num_agg[col.replace('nth2', 'last')] - test_num_agg[col]) / (test_num_agg[col] + 0.000001)
        test_num_agg.assign(**new_cols)
    # SUM Lag features
    for col in num_features:
        # "divide by last" (first/last, nth1/last, nth2/last)
        if PCT_CALCULATION_DIVIDE_BY_LAST == True:
            new_cols[col + '_divLast_SUM'] = test_num_agg[col + '_first_div_last'] + \
                test_num_agg[col + '_nth1_div_last'] + \
                test_num_agg[col + '_nth2_div_last']
            #test_num_agg[col + '_divLast_SUM'] = test_num_agg[col + '_first_div_last'] + test_num_agg[col + '_nth1_div_last'] + test_num_agg[col + '_nth2_div_last']
            # >test_num_agg[col + '_avgLast_SUM'] = test_num_agg[col + '_divLast_SUM'] / 3

        # "divide by next" (first/nth1, nth1/nth2, nth2/last)
        if PCT_CALCULATION_DIVIDE_BY_NEXT == True:
            # test_num_agg[col + '_divNext_SUM'] = test_num_agg[col + '_first_div_nth1'] + test_num_agg[col + '_nth1_div_nth2'] + test_num_agg[col + '_nth2_div_last']
            new_cols[col + '_divNext_SUM'] = test_num_agg[col + '_first_div_nth1'] + \
                test_num_agg[col + '_nth1_div_nth2'] + \
                test_num_agg[col + '_nth2_div_last']
            # >test_num_agg[col + '_avgNext_SUM'] = test_num_agg[col + '_divNext_SUM'] / 3
        test_num_agg.assign(**new_cols)
    print('4) Calculated percent change and SUM of pecent change features (first div last, ...)')
    gc.collect()

#     test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])   ### ORIG
    # TRY other options
    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(
        ['count', 'last', 'nunique', 'first'])  # TRY other options
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    print('5) Calculated test_cat_agg (e.g. count,last,nunique,first)')

    # >df = pd.concat([test_num_agg, test_cat_agg], axis=1)   ### ORIG
    df = pd.concat([test_num_agg, test_cat_agg], axis=1)

    # >del test_num_agg, test_cat_agg   ### ORIG
    del test_num_agg, test_cat_agg  # ORIG
    gc.collect()
    print('Shape after engineering', df.shape)

    #Num_Nan = df.isna().sum().sum()
    #print('Num_Nan: ' + str(Num_Nan))
    df = df.fillna(NAN_VALUE)
    return df


## Cross Validation

In [48]:
% % time
for i in ['test', 'train'] if INFERENCE else ['train']:
    print()
    print('Loading ' + str(i))
    print()

    df = pd.read_parquet(
        f'../input/amex-data-integer-dtypes-parquet-format/{i}.parquet')

    if i == 'train' and DROP_SOME_TRAIN_DATA == True:
        print('DROP_SOME_TRAIN_DATA')
        orig_length = len(df)
        print(str(orig_length))
        cutoff_point = orig_length // 2  # Drop the first half of the data
        print(str(cutoff_point))
        df = df.iloc[cutoff_point:]
        df.reset_index(inplace=True, drop=True)

    if USE_ALL_FEATURES == True:
        print('USE ALL FEATUTRES (cols): ' + str(len(df.columns)))
    else:
        # Just use SELECT features (not enough RAM for all features)
        print('USE SELECT FEATURES (cols): ')
        cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117",
                        "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
        cat_cols = ['Balance 30', 'Balance 38', 'Delinquency 63', 'Delinquency 64', 'Delinquency 66', 'Delinquency 68',
                    'Delinquency 114', 'Delinquency 116', 'Delinquency 117', 'Delinquency 120', 'Delinquency 126', 'Target']
        cols_to_keep = features_to_use + cat_cols + \
            cat_features + ['customer_ID', 'S_2']
        cols_to_drop = [col for col in df.columns if (col not in cols_to_keep)]
        # DROP R_ cols TO SAVE RAM for this test
        print('orig num cols: ' + str(len(df.columns)))
        df.drop(columns=cols_to_drop, inplace=True)
        gc.collect()
        print('NEW num cols: ' + str(len(df.columns)))

    df = process_and_feature_engineer(df)

    if i == 'train':
        train = df
    else:
        test = df
    print(f" Final {i} shape: {df.shape}")

    del df  # , df_avg, df_min, df_max, cid, last
    gc.collect()

# # Load dataset
# train_data = tf.data.experimental.make_csv_dataset(
#     'train.csv',
#     batch_size=32,
#     label_name='label',
#     num_epochs=1)

# # Train LightGBM model
# model = lgb.LGBMModel()
# model.fit(train_data, eval_set=validation_data, ...)

# This lets you leverage TensorFlow for reading and batching data, while using LightGBM for the actual training of the model. Hope that helps!



Loading test

USE SELECT FEATURES (cols): 
orig num cols: 190
NEW num cols: 148
135
VALUE_FOR_MONTH_NTH1: -9
1) Calculated _nth1
VALUE_FOR_MONTH_NTH2: -5
2) Calculated _nth2
3) Calculated test_num_agg (e.g. first,last,min,max,mean,median,std)


KeyError: 'P_2_first_div_last'

In [ ]:
% % time
# Cross-validation of the classifier

# - - - - - lgb approach...
# Label encode categorical features
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
cat_features = [f"{cf}_last" for cf in cat_features]
for cat_col in cat_features:
    encoder = LabelEncoder()
    train[cat_col] = encoder.fit_transform(train[cat_col])
    test[cat_col] = encoder.transform(test[cat_col])

if USE_LGB_WITH_DART == True:
    print('Training with DART')
else:
    print('Training WITHOUT DART')
# - - - - -

features = [f for f in train.columns if f != 'customer_ID' and f != 'target']


def my_booster(random_state=1, n_estimators=1200):
    return LGBMClassifier(n_estimators=n_estimators,
                          learning_rate=0.03, reg_lambda=50,
                          min_child_samples=2400,
                          num_leaves=95,
                          colsample_bytree=0.19,
                          max_bins=511, random_state=random_state)


print(f"{len(features)} features")
score_list = []
y_pred_list = []

ft_importance = pd.DataFrame(index=train.columns)  # ADD

kf = StratifiedKFold(n_splits=5)
# kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
# for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, target)): ### ORIG train[CFG.target])):
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, target)):
    X_tr, X_va, y_tr, y_va, model = None, None, None, None, None
    start_time = datetime.datetime.now()
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = target[idx_tr]
    y_va = target[idx_va]

    ### model = my_booster()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        # - - - - - lgb approach...
#         x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
#         y_train, y_val = target[CFG.target].iloc[trn_ind], target[CFG.target].iloc[val_ind]
        x_train = train.iloc[idx_tr][features]
        x_val = train.iloc[idx_va][features]
        y_train = target[idx_tr]
        y_val = target[idx_va]

        lgb_train = lgb.Dataset(
            x_train, y_train, categorical_feature=cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_features)
        model = lgb.train(
            params=params,
            train_set=lgb_train,
            num_boost_round=1500,  # WAS 10500
            valid_sets=[lgb_train, lgb_valid],
            early_stopping_rounds=100,  # WAS 100
            verbose_eval=200,
            feval=lgb_amex_metric
        )
        gc.collect()
        # - - - - -
#         model.fit(X_tr, y_tr,
#                   eval_set = [(X_va, y_va)],
#                   eval_metric=[lgb_amex_metric],
#                   callbacks=[log_evaluation(100)])

    X_tr, y_tr = None, None
    # ORIG   y_va_pred = model.predict_proba(X_va, raw_score=True)
    y_va_pred = model.predict(X_va, raw_score=True)
    score = amex_metric(y_va, y_va_pred)
    # >n_trees = model.best_iteration_
    #if n_trees is None: n_trees = model.n_estimators
    n_trees = 0
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | {str(datetime.datetime.now() - start_time)[-12:-7]} |"
          f" {n_trees:5} trees |"
          f"                Score = {score:.5f}{Style.RESET_ALL}")
    score_list.append(score)
    # ft_importance["Importance_Fold"+str(fold)]=model.feature_importances_   ### ADD

    if INFERENCE:
        # ORIG   y_pred_list.append(model.predict_proba(test[features], raw_score=True))
        y_pred_list.append(model.predict(test[features], raw_score=True))

    if ONLY_FIRST_FOLD:  # we only want the first fold
        print('RUN FIRST FOLD ONLY')
        break
    gc.collect()

gc.collect()
print(f"{Fore.GREEN}{Style.BRIGHT}OOF Score:                       {np.mean(score_list):.5f}{Style.RESET_ALL}")


# Derivative of features

---

In [ ]:
from unittest import skip
import numpy as np
import pdb
import traceback
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_classif

gc.collect()
train[features].drop('S_2', axis=1)
# compute gradient of each feature
# pdb.set_trace()
try:
    gradients = np.gradient(train[features], axis=1)
    scores = mutual_info_classif(gradients.T, target)
except TypeError as e:
    print(e)
    traceback.print_exc()

m_infos = scores[features]


In [ ]:
print(scores)


In [ ]:
from optuna import trial, study, create_study
import optuna
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from itertools import combinations
import os
import gc
import joblib
import random
import warnings
import itertools
import scipy as sp
import numpy as np
import pandas as pd
from tqdm import tqdm
import xgboost as xgb
import lightgbm as lgb
warnings.filterwarnings('ignore')
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


class IdealModel:
    # Implement model type and assign score to each
    def __init__(self, target, folds=5, input_dir='./input/amex-fe'):

        self.folds = folds
        self.input_dir = input_dir
        self.target = target

    def objective_cross(self, trial):
        cv = StratifiedKFold(n_splits=self.folds,
                             shuffle=True, random_state=1121218)

        cv_scores = np.empty(cv.n_splits)
        for fold_idx, (idx_tr, idx_va) in enumerate(cv.split(range(len(train)))):
            train_data, valid_data = train.iloc[idx_tr][features], train.iloc[idx_va][features]
            y_train, y_valid = self.target[idx_tr], self.target[idx_va]

            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=UserWarning)
                # - - - - - lgb approach...
        #         x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        #         y_train, y_val = target[CFG.target].iloc[trn_ind], target[CFG.target].iloc[val_ind]
                x_train = train.iloc[idx_tr][features]
                x_val = train.iloc[idx_va][features]
                y_train = target[idx_tr]
                y_val = target[idx_va]

                lgb_train = lgb.Dataset(
                    x_train, y_train, categorical_feature=cat_features)
                lgb_valid = lgb.Dataset(
                    x_val, y_val, categorical_feature=cat_features)
                model = lgb.train(
                    params=params,
                    train_set=lgb_train,
                    num_boost_round=10500,  # WAS 10500
                    valid_sets=[lgb_train, lgb_valid],
                    early_stopping_rounds=100,  # WAS 100
                    verbose_eval=200,
                    feval=lgb_amex_metric
                )
                gc.collect()

            X_tr, y_tr = None, None
            # ORIG   y_va_pred = model.predict_proba(X_va, raw_score=True)
            y_va_pred = model.predict(X_va, raw_score=True)
            score = amex_metric(y_va, y_va_pred)

            #cv_scores[idx] = log_loss(y_test, preds)
        return np.mean(cv_scores)

    def objective(self, trial, X, y, features):
        param_grid = {
            "device_type": trial.suggest_categorical("device_type", ['gpu']),
            "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
            "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
            "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
            "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
            "bagging_fraction": trial.suggest_float(
                "bagging_fraction", 0.2, 0.95, step=0.1
            ),
            "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
            "feature_fraction": trial.suggest_float(
                "feature_fraction", 0.2, 0.95, step=0.1
            ),
        }

        model = lgb.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X,
            y,
            eval_set=[(X, y)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                # LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        y_va_pred = model.predict(X_test, raw_score=True)

        return np.mean(cv_scores)

    def seed_everything(seed):
        random.seed(seed)
        np.random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)

    # def read_data(self):
    #     train = pd.read_parquet(self.input_dir + 'train_fe_plus_plus.parquet')
    #     test = pd.read_parquet(self.input_dir + 'test_fe_plus_plus.parquet')
    #     return train, test

    def amex_metric(y_true, y_pred):
        # count of positives and negatives
        n_pos = y_true.sum()
        n_neg = y_true.shape[0] - n_pos

        # sorting by descring prediction values
        indices = np.argsort(y_pred)[::-1]
        preds, target = y_pred[indices], y_true[indices]

        # filter the top 4% by cumulative row weights
        weight = 20.0 - target * 19.0
        cum_norm_weight = (weight / weight.sum()).cumsum()
        four_pct_filter = cum_norm_weight <= 0.04

        # default rate captured at 4%
        d = target[four_pct_filter].sum() / n_pos

        # weighted gini coefficient
        lorentz = (target / n_pos).cumsum()
        gini = ((lorentz - cum_norm_weight) * weight).sum()

        # max weighted gini coefficient
        gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

        # normalized weighted gini coefficient
        g = gini / gini_max

        return 0.5 * (g + d)

    def amex_metric_np(preds, target):
        indices = np.argsort(preds)[::-1]
        preds, target = preds[indices], target[indices]
        weight = 20.0 - target * 19.0
        cum_norm_weight = (weight / weight.sum()).cumsum()
        four_pct_mask = cum_norm_weight <= 0.04
        d = np.sum(target[four_pct_mask]) / np.sum(target)
        weighted_target = target * weight
        lorentz = (weighted_target / weighted_target.sum()).cumsum()
        gini = ((lorentz - cum_norm_weight) * weight).sum()
        n_pos = np.sum(target)
        n_neg = target.shape[0] - n_pos
        gini_max = 10 * n_neg * \
            (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)
        g = gini / gini_max
        return 0.5 * (g + d)


In [ ]:
study = create_study(direction="maximize")

ideal_model = IdealModel(target)

study.optimize(ideal_model.objective_cross(), n_trials=20, timeout=600)

pruned_trials = [t for t in study.trials if t.state ==
                 optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state ==
                   optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


# Graphing Hyperparameter Results

---

In [ ]:
if INFERENCE:
    sub = pd.DataFrame({'customer_ID': test.index,
                        'prediction': np.mean(y_pred_list, axis=0)})
    sub.to_csv(f'{file_name}_{CV_score}', index=False)
    display(sub)
